In [1]:
import time
from functools import reduce, wraps

## decorat function

In [2]:
def timecalc(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"[function]: {func.__name__}, [time]: {end - start}ms")
        return result
    return wrapper

In [3]:
@timecalc
def multiple(l):
    return reduce(lambda x,y: x+y**2, l)

In [4]:
multiple([i for i in range(62535)])

[function]: multiple, [time]: 0.016530990600585938ms


81515048357435

In [5]:
@timecalc
def generator(min, max, step=1):
    return [i for i in range(min, max, step)]

@timecalc
def listgenerator(min, max, step=1):
    return list(range(min, max, step))

In [6]:
l1 = generator(0,1000000)
l2 = listgenerator(0,1000000)

[function]: generator, [time]: 0.03351902961730957ms
[function]: listgenerator, [time]: 0.012525081634521484ms


## decorat class

In [7]:
def class_decorator(cls, *args, **kwargs):
    print(args)
    print(kwargs)
    @wraps(cls)
    def wrapper(*args, **kwargs):
        print(f"create {cls.__name__} with args: {args} and kwargs: {kwargs}")
        return cls(*args, **kwargs)
    return wrapper

In [8]:
@class_decorator
class TestClass(object):
    """TestClass doc

    Args:
        object (_type_): _description_
    """
    def __init__(self, *args, **kwargs):
        print(f"init {self.__class__.__name__} with args: {args} and kwargs: {kwargs}")
        pass

()
{}


In [9]:
c = TestClass(123,321,test=321)

create TestClass with args: (123, 321) and kwargs: {'test': 321}
init TestClass with args: (123, 321) and kwargs: {'test': 321}


In [10]:
c.__doc__

'TestClass doc\n\n    Args:\n        object (_type_): _description_\n    '

In [11]:
def class_decorator(cls, *args, **kwargs):
    print(args)
    print(kwargs)
    cls_init = cls.__init__
    @wraps(cls)
    def wrapper(self, *args, **kwargs):
        print(f"create {self.__class__.__name__} with args: {args} and kwargs: {kwargs}")
        cls_init(*args, **kwargs)
    cls.__init__ = wrapper
    return cls

In [12]:
@class_decorator
class TestClass(object):
    """TestClass doc

    Args:
        object (_type_): _description_
    """
    def __init__(self, *args, **kwargs):
        print(f"init {self.__class__.__name__} with args: {args} and kwargs: {kwargs}")
        pass

()
{}


In [13]:
c = TestClass(123,321,test=321)

create TestClass with args: (123, 321) and kwargs: {'test': 321}
init int with args: (321,) and kwargs: {'test': 321}


In [14]:
c.__doc__

'TestClass doc\n\n    Args:\n        object (_type_): _description_\n    '

## timing all the function of class

In [15]:
def timer(cls):
    for attr_name, attr_value in cls.__dict__.items():
        if callable(attr_value):
            # 对每个方法添加日志功能
            original_method = attr_value
            @timecalc
            @wraps(original_method)
            def time_wrapper(*args, **kwargs):
                return original_method(*args, **kwargs)
            setattr(cls, attr_name, time_wrapper)
    return cls

In [16]:
@timer
class MyGenerator:
    def generator(self, min, max, step=1):
        return [i for i in range(min, max, step)]

    def listgenerator(self, min, max, step=1):
        return list(range(min, max, step))

In [17]:
g = MyGenerator()

In [18]:
l = g.generator(1,100000000)
l = g.listgenerator(1,100000000)

[function]: generator, [time]: 1.0958898067474365ms
[function]: listgenerator, [time]: 1.1724488735198975ms


## Class register

In [19]:
class FrameworkDatasets:
    def __new__(cls, *args, **kwargs):
        cls.__DATASET = {}
        return super().__new__(cls, *args, **kwargs)
    
    def dataset_register(self, force=False):
        def _register(cls):
            assert not cls.__name__ in self.__DATASET or force, Exception(f"{cls.__name__} already registed")
            self.__DATASET[cls.__name__] = cls
        return _register
    
    def build(self, dataset, *args, **kwargs):
        assert dataset in self.__class__.__DATASET, Exception(f"{dataset} not registed")
        return self.__class__.__DATASET[dataset](*args, **kwargs)
    

In [20]:
DATASETS = FrameworkDatasets()

In [21]:
@DATASETS.dataset_register()
class COCODataset:
    def __init__(self, *args, **kwargs):
        pass

@DATASETS.dataset_register()
class YOLODataset:
    def __init__(self, *args, **kwargs):
        pass

In [22]:
try:
    @DATASETS.dataset_register()
    class COCODataset:
        def __init__(self, *args, **kwargs):
            pass
except Exception as e:
    import traceback
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\25335\AppData\Local\Temp\ipykernel_118820\3717501706.py", line 3, in <module>
    class COCODataset:
  File "C:\Users\25335\AppData\Local\Temp\ipykernel_118820\281006387.py", line 8, in _register
    assert not cls.__name__ in self.__DATASET or force, Exception(f"{cls.__name__} already registed")
AssertionError: COCODataset already registed


In [23]:
@DATASETS.dataset_register(force=True)
class COCODataset:
    def __init__(self, *args, **kwargs):
        pass

In [24]:
DATASETS.build("COCODataset", path="./coco")